In [1]:
from functools import cached_property
import re
import numpy as np

TODAY = 'day13'
TEST_FILE_INPUT = f"./test_input_{TODAY}.txt"
FILE_INPUT = f"./input_{TODAY}.txt"


In [2]:
np.zeros([3,2])

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [3]:
class PatternBoard:
    def __init__(self, line_list):
        # line_list = list of lines parsed from input file
        self.line_list = line_list
        self.n_rows = len(self.line_list)
        self.n_cols = len(self.line_list[0])
        
    @cached_property
    def matrix(self):
        """Convert ./# board to 0/1 matrix"""
        M = np.zeros([self.n_rows, self.n_cols])
        
        for ii, line in enumerate(self.line_list):
            for jj, char in enumerate(line):
                if char == '#':
                    M[ii, jj] = 1
                
        return M
    
    def row_margins(self, matrix=None):
        """Get binary row sums"""
        if matrix is None:
            matrix = self.matrix
        return [sum(matrix[ii,:] * np.array([2 ** jj for jj in range(self.n_cols)])) for ii in range(self.n_rows)]

    def col_margins(self, matrix=None):
        """Get binary column sums"""
        if matrix is None:
            matrix = self.matrix
        return [sum(matrix[:,jj] * np.array([2 ** ii for ii in range(self.n_rows)])) for jj in range(self.n_cols)]
    

    def flip_bit(self, row, col):
        """Flip the bit at position (row, column) in self.matrix. Return a copy of the resulting matrix"""
        M = self.matrix.copy()
        M[row, col] = 1 - M[row, col]
        return M
    
    def is_axis_of_reflection(self, row_col_number, direction, matrix = None):
        """Check whether the line between row/column number <row_col_number> and <row_col_number +1> in matrix is an axis of reflection."""
        if matrix is None:
            matrix = self.matrix
        
        if direction == "row":
            value_list = self.row_margins(matrix)
        else:
            value_list = self.col_margins(matrix)
            
        # List of binary margins when reading to the left/up from the axis of reflection
        before_slice = value_list[:row_col_number]
        before_slice.reverse()
        
        # List of binary margins when reading to the right/down from the axis of reflection
        after_slice = value_list[row_col_number:]
        
        # It is a valid axis of reflection if all binary margins working away from the axis are equal
        # for as long as both numbers exist
        are_equal = [
            before_slice[ii] == after_slice[ii] f
            or ii in range(min(len(before_slice), len(after_slice)))
        ]
        return int(all(are_equal))
    
    def axes_of_reflection(self, matrix = None):
        """Find all axes of reflection in matrix.
        
        Returns a dictionary with  keys "row" and "col pointing to lists of 
        valid axes of row/column reflections
        """
        
        if matrix is None:
            matrix = self.matrix
            
        reflection_dict = {"row": [], "col": []}
        
        n_rows = matrix.shape[0]
        n_cols = matrix.shape[1]
            
        for ii in range(1, n_rows):
            if self.is_axis_of_reflection(ii, "row", matrix) == 1:
                reflection_dict["row"].append(ii)

        for ii in range(1, n_cols):
            if self.is_axis_of_reflection(ii, "col", matrix) == 1:
                reflection_dict["col"].append(ii)
                
        return reflection_dict
            
        

class PartA:
    def __init__(self, file_path):
        self.file_path = file_path
        
    @cached_property
    def boards(self):
        """Read all boards from input file"""
        boards = []
        
        current_board = []
        with open(self.file_path, 'r') as f:
            for line in f:
                line = line.rstrip('\n')
                if line == '':
                    boards.append(PatternBoard(current_board))
                    current_board = []
                else:
                    current_board.append(line)
                    
        boards.append(PatternBoard(current_board))
                
        return boards
    
    def solve(self):
        cumsum = 0
        for board in self.boards:
            reflection_dict = board.axes_of_reflection()
            cumsum += 100 * sum(reflection_dict["row"]) + sum(reflection_dict["col"])
                
        return cumsum

In [4]:
a_test = PartA(TEST_FILE_INPUT)

In [5]:
assert a_test.solve() == 405# NUMER HERE

In [6]:
a = PartA(FILE_INPUT)
a.solve()

27664

In [11]:
class PartB(PartA):
    def solve(self):
        cumsum = 0
        for board in self.boards:
            smudge_found = False
            reflection_dict_original = board.axes_of_reflection()
            
            # Find the first smudge and the corresponding new row/column axis of reflection
            for ii in range(board.n_rows):
                for jj in range(board.n_cols):
                    if not smudge_found:
                        new_matrix = board.flip_bit(ii, jj)
                        reflection_dict_new = board.axes_of_reflection(new_matrix)

                        # row/col indices that are new axes of reflection
                        row_diff = [
                            x for x in reflection_dict_new["row"] 
                            if x not in reflection_dict_original["row"]
                        ]
                        col_diff = [
                            x for x in reflection_dict_new["col"] 
                            if x not in reflection_dict_original["col"]
                        ]
                        
                        if len(row_diff) > 0:
                            smudge_found = True
                            cumsum += row_diff[0] * 100
                            
                        elif len(col_diff) > 0:
                            smudge_found = True
                            cumsum += col_diff[0]
                                  
        return cumsum
                
            

In [12]:
b_test = PartB(TEST_FILE_INPUT)

In [13]:
assert b_test.solve() == 400 # NUMBER HERE

In [14]:
b = PartB(FILE_INPUT)
b.solve()

33991